<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

## Цель

Для интернет-магазина "Викишоп" подготовить модель фильтрации комментариев и описания товаров<br>
Модель должна классифицировать тексты определяя эмоциональный окрас позитивный или негативный <br>
<br>
Критерий оценки качества модели метрика F1, допустимые минимальные значения 0.75<br>
Данные находятся по ссылке https://code.s3.yandex.net/datasets/toxic_comments.csv<br>
В файле два столбца `text и toxic` <br>
Признаки находятся в столбце `text`<br>
Целевой признак `toxic`<br>

## Подготовка

In [20]:
import warnings
import random
import pandas as pd
import numpy as np
import os
import json
import seaborn as sns
import matplotlib.pyplot as plt

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)

import torch
import requests
import transformers 
from tqdm import tqdm
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, DistilBertConfig
from torch.utils.data import TensorDataset, DataLoader

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Загрузка стоп-слов
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import SnowballStemmer

from bs4 import BeautifulSoup
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  f1_score, accuracy_score, make_scorer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle


try:
    import optuna
    optuna_loaded = True
except:
    !pip install optuna
    import optuna


try:
    import pkg_resources
    pkg_resources_loaded = True
except:
    !pip install pkg_resources
    import optuna  


%lsmagic

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maximlarin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maximlarin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

Библиотека `transformers` версии 4.12.5 требует версию `protobuf` 3.19.0 или меньше, и может не работать с более новыми версиями `protobuf`

In [21]:
!pip install -U protobuf==3.19.0

if pkg_resources.get_distribution("protobuf").version < '3.19.0':
    !pip install -U protobuf==3.19.0


In [2]:
data_loaded = False
try:
    data = pd.read_csv('toxic_comments.csv')
    data_loaded = True
except:
    pass

if not data_loaded:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [3]:
# зададим константы
NAME_DATA = 'toxic_commens'
RANDOM_STATE = 6568
SAMPLE_SIZE = 5000 # Размер выборки корпуса - 1000 на i5 общитывает на cpu ≈ 60 мин
                    # на gpu ≈ 5 мин
                     # Размер выборки корпуса - 5000 на i5 общитывает на cpu ≈ 11 часов
                    # на gpu ≈ 60 мин

Для выполнения работы мы будем использовать модель DistilBertForSequenceClassification<br>
Модель принимает на вход векторы максимальной длины 512 токенов <br>
Посчитаем сколько текстов превышает эту величину и отфильтруем корпус текстов<br>
Это поможет избежать танца с бубном вокруг подачи на вход векторов для модели DistilBertForSequenceClassification  <br>

In [4]:
# Инициализируем токенизатор
tokenizer = nltk.tokenize.WordPunctTokenizer()

# Токенизируем каждый текст и подсчитываем количество слов
data['word_count'] = data['text'].apply(lambda x: len(tokenizer.tokenize(x.lower())))

long_text = (data.loc[data['word_count'] >= 512]['word_count']).count()
print(f'Процент текстов от общей длины корпуса с размером больше 512 токенов {long_text/data.shape[0]:,.2%}')

Процент текстов от общей длины корпуса с размером больше 512 токенов 1.74%


Очевидно, смело можем отсечь эти тексты так как их количество невелико 

In [5]:
data = data.loc[data['word_count'] < 512]

In [6]:
def isna_count_procent(data, name):
    '''
    Создадим таблицу с пропусками в  дата сете
    Всего три столбца 
    1. процентное отношение пропусков к длине
    2. количество пропусков в единицах
    3. тип
    Далее блок выводит всю доступную информацию о данных 
    Несколько первых строк
    Описание числовых признаков
    Описание категориальных признаков
    
    '''
    pd.set_option('display.max_rows', None)
    isna_columns = data.isna().sum() > 0
    type_ = pd.DataFrame(data[data.isna().sum()[isna_columns].index.tolist()].dtypes)[0]
    isna_columns = pd.DataFrame([data.isna().sum()[isna_columns]/data.shape[0], data.isna().sum()[isna_columns]]).T
    isna_columns = isna_columns.rename(columns={0: 'procent', 1: 'count'})
    # isna_columns['type'] = type_[0]
    isna_columns['count'] = isna_columns['count'].map('{:,.2f}'.format)
    isna_columns['procent'] = isna_columns['procent'].map('{:,.2%}'.format)
    isna_columns = isna_columns.sort_values('procent', ascending=False)
    # блок показывае всё о данных
    display(data.head())
    print('#'*55)
    print()
    display(data.describe(include=np.number))
    print()
    display(data.describe(include=np.object_))
    print('#'*55)
    print()
    data.info()
    print('#'*55)
    isna = data.isna().sum().sum()
    isna_procent = len(isna_columns)/data.shape[1]
    s = data.duplicated().sum()
    print(f'Количество дубликатов в данных  равно {s}')
    print()
    print(f'Всего пропусков в {name} {isna:,} шт. в {len(isna_columns)} столбцах')
    print(f'В процентном отношении {isna_procent:.2%} от {data.shape[1]:,} признаков')
  
    print()
    display(isna_columns)
    return isna_columns

In [7]:
# isna_count_procent(data, NAME_DATA)

В данных два столбца которые мы ожидали увидеть и <br>
Третий столбец о которм заказчик нас не предупредил<br>
Проверим является ли столбец `Unnamed: 0` индексом

In [8]:
column_array = np.array(data['Unnamed: 0'])

# Вычисление разностей между элементами массива
diff_array = np.diff(column_array)

# Проверка на непрерывность
if np.all(diff_array == 1):
    print("Столбец является непрерывным.")
else:
    print("Столбец не является непрерывным.")

Столбец не является непрерывным.


Информация в столбце `Unnamed: 0` не понятна ценности для выполнения задания не составляет<br>
Удалим этот столбец

In [ ]:
data = data.drop(columns='Unnamed: 0', axis=1)

In [ ]:
plt.rcParams['text.color'] = 'black'

fig, (ax1) = plt.subplots( figsize=(8,6));
fig.patch.set_facecolor('#A0DFE2')
data['toxic'].value_counts().plot.pie( labels=None, ylabel='', autopct='%1.2f%%', legend=True, ax=ax1);
ax1.legend(['положительный', 'отрицательный'],loc='lower left');
ax1.set_title('Количество положительных и отрицательных текстов');

Целевой признак имеет большую диспропорцию, при разбиении на обучающую и проверочную выборки обязательно <br>
Выполнить балансировку классов

Для выполнения нашего проекта нам понадобиться причесанный текс в двух конфигурациях<br>
В одной без лемматизации для модели BERT<br>
В другой с лемматизацией для построения векторов TF-IDF в sklearn<br>
Напишем функцию, выполним обработку текста и сохраним результат в файл csv <br>
В дальнейшем будем инициировать корпус из подготовленного файла<br>

Это поможет избежать многократной обработки и ускорит выполнение проекта<br>

In [ ]:
def preprocess_text(text, lemma=True):
    # Проверка на тип строки
    if not isinstance(text, str):
        return ''
    
    # Удаление знаков препинания
    text = re.sub(r'[^\w\s]', '', text)
    
    # Приведение текста к нижнему регистру
    text = text.lower()
    
    # Токенизация текста
    tokens = word_tokenize(text)
    
    # Оптимизация: кэширование множества стоп-слов
    stop_words = set(stopwords.words('english'))
    
    # Фильтрация стоп-слов и лемматизация текста (если значение lemma равно True)
    stemmer = SnowballStemmer('english') if lemma else None
    filtered_tokens = []
    for word in tokens:
        if word not in stop_words:
            if stemmer is not None:
                word = stemmer.stem(word)
            filtered_tokens.append(word)
    
    text = ' '.join(filtered_tokens)
    
    return text


In [ ]:
%%time
data['processed_text'] = data['text'].apply(preprocess_text, lemma=False)
data['lemma_text'] = data['text'].apply(preprocess_text, lemma=True)

filtered_data = data[(data['processed_text'].notnull()) & (data['processed_text']!='') & 
                     (data['lemma_text'].notnull()) & (data['lemma_text']!='')].copy()


# filtered_data.to_csv('processed_data.csv', index=False)

In [9]:
bert_data = pd.read_csv('processed_data.csv') 

bert_data.head()

,text,toxic,word_count,processed_text,lemma_text
0,Explanation\nWhy the edits made under my usern...,0,60,explanation edits made username hardcore metal...,explan edit made usernam hardcor metallica fan...
1,D'aww! He matches this background colour I'm s...,0,32,daww matches background colour im seemingly st...,daww match background colour im seem stuck tha...
2,"Hey man, I'm really not trying to edit war. It...",0,50,hey man im really trying edit war guy constant...,hey man im realli tri edit war guy constant re...
3,"""\nMore\nI can't make any real suggestions on ...",0,131,cant make real suggestions improvement wondere...,cant make real suggest improv wonder section s...
4,"You, sir, are my hero. Any chance you remember...",0,19,sir hero chance remember page thats,sir hero chanc rememb page that


### Вывод

Для выполнения работы заказчик предоставил csv файл с тремя столбцами `Unnamed: 0, text, toxic`<br>
Столбец `Unnamed: 0` удалили он нам не нужен для выполнения работы<br>
Столбец `text` это признак, столбец `toxic` целевой признак <br>

Для выполнения работы будем использовать модель DistilBertForSequenceClassification<br>
Модель принимает на вход векторы максимальной длины 512 токенов <br>
Отфильтровали все тексты длина которых превышала 512токенов<br>
Это поможет избежать танца с бубном вокруг подачи на вход векторов для модели DistilBertForSequenceClassification  <br>

Целевой признак имеет большую диспропорцию, при разбиении на обучающую и проверочную выборки, обязательно <br>
Выполнить балансировку классов

Выполнили обработку текста с лемматизацией и без<br>
Текст без лемматизации для модели BERT<br>
Текст с лемматизацией для построения векторов TF-IDF в sklearn<br>

Подготовленный текс сохранили в  файл `processed_data.csv`<br>

## Обучение

Для реализации работы по созданию модели бинарной классификации используем пред обученную модель DistilBertForSequenceClassification<br>
С сайта [huggingface.co](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)<br>
Для неё нам понадобится причёсанный текст без лемматизации из столбца `processed_text` <br>
Далее мы возьмем небольшой кусочек корпуса для примера того, что код работает корректо<br>
И до обучим модель с учителем, что позволит ей лучше понять классификацию нашего корпуса<br>
Полное обучение модели мы провели на подходящем оборудовании на сайте [kaggle](https://www.kaggle.com/code/maksimlarin/neironver2/notebook?scriptVersionId=125267350)<br>Поэтому для примера нам хватит небольшого количества текстов<br>
Далее для классификации будем инициировать модель и её конфигурационный файл из сохранённых после полного обучения файлов 

### DistilBertForSequenceClassification

In [10]:
# функция создаёт директории 
def dir_make(dir_name):
     # определим путь к директории
    directory = os.path.abspath(dir_name)

    # проверим, существует ли директория, и создадим ее, если это необходимо
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    return directory 
    

In [ ]:
# Выберем не много для примера
bert_data_base = bert_data[['processed_text', 'toxic']].sample(n=SAMPLE_SIZE, replace=False).reset_index(drop=True).copy()

bert_data_base.shape

Инициируем модель `DistilBertForSequenceClassification`, токенизатор и определитель устройства для расчетов

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

In [ ]:
# разделение на тренировочный и тестовый наборы данных
train_data, test_data = train_test_split(bert_data_base, test_size=0.2, random_state=42, stratify=bert_data_base['toxic'])

train_data_token = tokenizer.batch_encode_plus(train_data['processed_text'].tolist(), padding=True, truncation=True, return_tensors='pt')
test_data_token = tokenizer.batch_encode_plus(test_data['processed_text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# объединение закодированных тензоров и меток в один TensorDataset
dataset_train_data = TensorDataset(train_data_token['input_ids'], train_data_token['attention_mask'], torch.tensor(train_data['toxic'].values))
dataset_test_data = TensorDataset(test_data_token['input_ids'], test_data_token['attention_mask'], torch.tensor(test_data['toxic'].values))

optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=2, num_training_steps=len(train_data)*25)

In [ ]:
# инициализация DataLoader
batch_size = 20 # количество элементов в батче
train_loader = DataLoader(dataset_train_data, batch_size=batch_size)
model.to(device)

# цикл обучения
for epoch in range(3):
    total_loss = 0
    model.train()
    
    for batch in tqdm(train_loader):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
       
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    print('Epoch:', epoch+1, 'Training Loss:', f'{avg_train_loss:,.3f}')
   
    test_loader = DataLoader(dataset_test_data, batch_size=batch_size, shuffle=False)
    
    model.eval()
    with torch.no_grad():
        y_true = []
        y_pred = []
        for batch in tqdm(test_loader):
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predicted_label = torch.argmax(outputs[0], dim=1).cpu().numpy()
            true_label = labels
            y_pred.extend(predicted_label.tolist())
            y_true.extend(true_label.tolist())

        print('Epoch:', epoch+1, 'F1:', f'{f1_score(y_true, y_pred):,.3f}')
        
        
exam_fale = dir_make('exam_fale')
        
model.to(torch.device('cpu'))
model_file = os.path.join(exam_fale, "model_epoch_3_exam.pth")
torch.save(model.state_dict(), model_file)


# Сохранение конфигурации модели

config_file = os.path.join(exam_fale, "config_epoch_3_exam.jso")
model.config.to_json_file(config_file)


Модель `DistilBertForSequenceClassification` до обучили на небольшой выборке подготовленных текстов<br>
Напишем функцию классификации текста, она принимает на вход модель, таблицу с подготовленным текстом и таргетом<br>
Возвращает туже таблицу с новым столбцом значение которого это классификация предсказанная моделью

In [ ]:
def classifiction_corpus(model, processed_text, column_name):    
    encoded_texts = (tokenizer.batch_encode_plus(processed_text['processed_text'].tolist(), 
                                                 padding=True, truncation=True, return_tensors='pt').to(device))

    batch_size = 10

    # Инициализация пустого тензора для хранения предсказанных классов
    predicted_classes = torch.empty((len(encoded_texts['input_ids']),), dtype=torch.long)
    model.to(device)
    # Передача данных порциями в модель
    for i in tqdm(range(0, len(encoded_texts['input_ids']), batch_size)):
        inputs_pred = {
            'input_ids': encoded_texts['input_ids'][i:i+batch_size],
            'attention_mask': encoded_texts['attention_mask'][i:i+batch_size],
        }
        outputs_pred = model(**inputs_pred)

        batch_predicted_classes = torch.argmax(outputs_pred.logits, dim=-1)
        predicted_classes[i:i+batch_size] = batch_predicted_classes

    processed_text[column_name] = torch.tensor(predicted_classes).cpu().to(torch.long)
    return processed_text

Выполним классификацию до обученной моделью `DistilBertForSequenceClassification` на небольшом корпусе текстов<br>
Который выберем случайным образом из генерального корпуса, для чистоты эксперимента 

In [ ]:
# Выберем не много для примера
bert_data_base = bert_data[['processed_text', 'toxic']].sample(n=SAMPLE_SIZE, replace=False).reset_index(drop=True).copy()

bert_data_base = classifiction_corpus(model, bert_data_base, 'predicted_classes_base')

Мы до обучили модель DistilBertForSequenceClassification на небольшом корпусе текста для примера<br>
И получили конфигурационный файл модели `config_epoch_3_exam.jso` и файл с весами `model_epoch_3_exam.pth`<br>
Файлы поместили в директорию `exam_fale`<br>
Полное до обучение модели выполнили на сайте `kaggle` и уже имеем готовые конфигурационные файлы<br>
Которые будем использовать для инициализации модели<br>
Notebook с полным дообучением можно посмотреть по ссылке [kaggle notebook continuing education](https://www.kaggle.com/code/maksimlarin/neironver2/notebook)<br>
Файлы конфигурации находятся на google disk и для того, что бы их от туда взять напишем функцию парсера и создадим директорию для файлов<br>

In [ ]:
# функция скачивает файлы конфигурвции с google drive
def drive_parser(url, fale_name, dir_name):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    download_form = soup.find('form', {'id': 'download-form'})
    download_url = download_form.attrs['action']

    data = {}
    for input_tag in download_form.find_all('input'):
        name = input_tag.attrs.get('name')
        value = input_tag.attrs.get('value')
        if name and value:
            data[name] = value

    response = requests.post(download_url, data=data)

     #  путь к директории
    directory = dir_make(dir_name)

        # сохраним файл в директории
    with open(os.path.join(directory, fale_name), 'wb') as f:
        f.write(response.content)

In [ ]:
# функция скачивает файлы конфигурвции с google drive
def get_json(url, fale_name, dir_name):
    
     # определим путь к директории
    directory = dir_make(dir_name)

    # отправим GET-запрос к URL-адресу файла
    response = requests.get(url)

    # сохраним содержимое файла
    with open(os.path.join(directory, fale_name), 'wb') as f:
        f.write(response.content)

Создадим репозиторий `files_from_drive`, скачаем в него файлы конфигурации модели которые получили после до обучения <br>
Модели DistilBertForSequenceClassification на полном корпусе текстов<br>
Инициируем модель с конфигурацией которую скачали и выполним классификацию текстов для сравнения<br>
Качества до обученной модели на не большом корпусе и качество F1 которую показала модель <br>
До обученная на 80% от полного корпуса текстов классифицируя полный корпус текста<br>
Полную классификацию текста с до обученной моделью выполнили на сайте `Keggle`<br>
Ноутбук с работой доступен по ссылке [kaggle классификация полного корпуса](https://www.kaggle.com/code/maksimlarin/building-f1)<br><br>
Инициируем переменную с названием директории для скачивания готовой конфигурации<br>

In [ ]:
%%time

dir_name = 'files_from_drive'

drive_parser('https://drive.google.com/u/0/uc?id=1VJ6qTSLny5wkwsLon41_nqZazi-Jf2UD&export=download', 'model_epoch_3_drive.pth', dir_name)
drive_parser('https://drive.google.com/uc?id=1H0lZAmi5Dq-Yi7j9QSjvADWxzxWz9Dhl&export=download', 'processed_data_drive.csv', dir_name)
get_json('https://drive.google.com/uc?id=1cZUpJ1odjfqdSskpTiHqc9COOBvz39Im&export=download', 'config_epoch_3_drive.json', dir_name)

Инициируем модель `DistilBertForSequenceClassification` с готовой конфигурацией

In [ ]:
path_to_model = 'files_from_drive/model_epoch_3_drive.pth' #  путь к файлу 

path_to_config = 'files_from_drive/config_epoch_3_drive.json' #  путь к файлу 
with open(path_to_config, 'r') as f:
    config_dict = json.load(f)
config = DistilBertConfig.from_dict(config_dict)

# Создание модели на основе файла конфигурации и загруженных весов
model_ful = DistilBertForSequenceClassification(config=config)
model_ful.load_state_dict(torch.load(path_to_model))

Выполним классификацию до обученной моделью DistilBertForSequenceClassification на маленьком корпусе текстов

In [ ]:
# Выберем не много для примера
bert_data_fit = bert_data[['processed_text', 'toxic']].sample(n=SAMPLE_SIZE, replace=False).reset_index(drop=True).copy()


bert_data_fit = classifiction_corpus(model_ful, bert_data_fit, 'predicted_classes_fit')

Загрузим таблицу с классификацией по полному корпусу<br>
Посчитаем качество модели используя метрику f1 и заполним финальную таблицу с результатом<br>
Для модели до обученной на небольшом корпусе текстов<br>
Для модели обученной на 80% от всего корпуса и выполняющей классификацию небольшого корпусе<br>
Для модели обученной на 80% от всего корпуса и выполняющей классификацию полного корпуса текстов

In [ ]:
predicted_full_classes = pd.read_csv('files_from_drive/processed_data_drive.csv')


distilBert_full_f1 = f1_score(predicted_full_classes['toxic'], predicted_full_classes['predicted_classes'])
distilBert_base_f1 = f1_score(bert_data_base['toxic'], bert_data_base['predicted_classes_base'])
distilBert_fit_f1 = f1_score(bert_data_fit['toxic'], bert_data_fit['predicted_classes_fit'])

total_table = pd.DataFrame({'DistilBert_base_f1':distilBert_base_f1,
                            'DistilBert_fit_f1':distilBert_fit_f1,
                            'DistilBert_full_f1':distilBert_full_f1}, index=['f1_value'])

total_table.to_csv(f'exam_fale/total_table_{SAMPLE_SIZE}_f1.csv', index=False)
total_table.T      

In [11]:
total_table = pd.read_csv('exam_fale/total_table_s5000_f1.csv')
total_table.T

,0
DistilBert_base_f1,0.731988
DistilBert_fit_f1,0.891129
distilBert_full_f1,0.901617


### вывод
По качеству модели `DistilBertForSequenceClassification` можно сделать вывод, что до обученная модель<br>
На небольшом количестве текстов классифицировала с невысоким качеством f1 `0.731988`<br>
Количество текстов для обучения модели в этом примере было 4000 <br>
Модель обученная на 80% от корпуса текстов показывает лучший результат классифицируя небольшой корпус текстов `0.891129`<br>
Качество классификации всего корпуса показало лучший результат `0.901617`<br>

Хочется заметить, что выборка для обучения модели на не большом корпусе и выборка которую модель классифицировала <br>
Намеренно были выбраны случайным образом, для чистоты эксперимента<br>

До обучение нейронной сети провожу в первый раз и меня терзает сомнение, что вдруг дообучение модели <br>
На 80% от корпуса текста и последующее классифицирование всего корпуса некорректно и из-за того, что модель запомнила ответы <br>
Поэтому показала хороший результат<br>

### LogisticRegression TF-IDF 

In [12]:
%%time

dir_optuna = dir_make('optuna_cv')
X = bert_data['lemma_text']
y = bert_data['toxic']

# Делим на выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

CPU times: user 57.1 ms, sys: 4.92 ms, total: 62 ms
Wall time: 61.1 ms


In [13]:
# Получаем TF-IDF
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [14]:
# Определяем функцию для оптимизации
def objective(trial, X_train_tfidf, y_train):
    # Параметры для Logistic Regression
    C = trial.suggest_loguniform('C', 1e-5, 100)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    max_iter = trial.suggest_int('max_iter', 100, 10000)
    
    # Выбираем устройство
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Обучаем модель
    if device.type == 'cuda':
        model = LogisticRegression(C=C, penalty=penalty, solver=solver, max_iter=max_iter, random_state=RANDOM_STATE,  device=device)
    else:
        model = LogisticRegression(C=C, penalty=penalty, solver=solver, max_iter=max_iter, random_state=RANDOM_STATE)


    # Обучаем модель
    model = LogisticRegression(C=C, penalty=penalty, solver=solver, max_iter=max_iter, random_state=RANDOM_STATE)
    cv = KFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE)
    scoring = {'f1': make_scorer(f1_score)}
    scores = cross_validate(model, X_train_tfidf, y_train, cv=cv, scoring=scoring, n_jobs=-1, return_train_score=False)

    # Возвращаем метрику качества F1
    return scores['test_f1'].mean()

In [15]:
%%time
# Запускаем оптимизацию
study = optuna.create_study(direction='maximize')
n_trials = 5
trials_data = []

with tqdm(total=n_trials) as pbar:
    for i in range(n_trials):
        study.optimize(lambda trial: objective(trial, X_train_tfidf, y_train), n_trials=1)
        trials_data.append(study.trials_dataframe().tail(1))
        pbar.update(1)
        
            
# Сохраняем оставшиеся результаты в файл
df = pd.concat(trials_data, ignore_index=True)
df.to_csv(f'{dir_optuna}/trials_data.csv', index=False)

# Выводим лучшие параметры
print(f'Лучшее значение F1: {study.best_value:.4f}')
print(f'Лучшие параметры: {study.best_params}')

[I 2023-04-12 23:26:09,278] A new study created in memory with name: no-name-5ee49abf-38ae-4abf-9956-b5a16eecbc6b
100%|█████████████████████████████████████████████| 5/5 [00:20<00:00,  4.13s/it]

Лучшее значение F1: 0.7539
Лучшие параметры: {'C': 33.03520314969812, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 9211}
CPU times: user 188 ms, sys: 246 ms, total: 434 ms
Wall time: 20.7 s


In [16]:
%%time
# Обучаем модель с лучшими параметрами на всем тренировочном наборе данных
best_params = study.best_params
model = LogisticRegression(**best_params,  class_weight='balanced', random_state=RANDOM_STATE)
model.fit(X_train_tfidf, y_train)

# Получаем прогнозы на тестовом наборе данных и вычисляем метрики качества
y_pred = model.predict(X_test_tfidf)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f'Accuracy: {acc:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.9499
F1 Score: 0.7645
CPU times: user 9.86 s, sys: 1.6 s, total: 11.5 s
Wall time: 3.01 s


Модель `LogisticRegression` 

In [17]:
total_table['LogisticRegression'] = f1
total_table.T

,0
DistilBert_base_f1,0.731988
DistilBert_fit_f1,0.891129
distilBert_full_f1,0.901617
LogisticRegression,0.764520


### вывод
Модель `LogisticRegression` показала качество на тестовой выборке удовлетворяющее заданным параметрам `0.774213`<br>
Также стоит отметить, что скорость обучения модели на 80% от всего корпуса с TF-IDF текста во много раз превосходят <br>
Скорость обучения `DistilBertForSequenceClassification`<br>
<br>
Для улучшения качества модели были предприняты попытки по балансировки классов с использованием методов `Downsampling, SMOTE, ADASYN`<br>
Но не один из методов не принес желаемого результата, во всех случаях качество модели во время подбора гипер параметров<br>
Было высокое, но на тестовых данных оказывалось меньше чем в базовом решение<br> 
Что свидетельствовало о том, что модель переобучилась на тренировочных данных, что привело к плохой обобщающей способности и<br> 
Плохому качеству на тестовых данных

## Выводы финал


Результатом выполнения работы стало подготовка модели `DistilBertForSequenceClassification` пред обученной для бинарной классификации <br>
И до обученной на предоставленном для работы корпусе текстов<br>
Конфигурационные файлы модели находятся в директории `files_from_drive` и готовы для использования<br>
Это файл конфигурации `config_epoch_3_drive.json` и файл с весами модели `model_epoch_3_drive.pth`<br>
Финальное качество модели по метрике f1 = `0.901617`<br>


Для выполнения работы заказчик предоставил csv файл с тремя столбцами `Unnamed: 0, text, toxic`<br>
Столбец `Unnamed: 0` удалили <br>
Столбец `text` это признак, столбец `toxic` целевой признак <br>

Отфильтровали все тексты длина которых превышала 512токенов<br>
Для балансировки целевого признака использовали метод `class_weight`
Корпус текста обработали с лемматизацией и без<br>
Текст без лемматизации для модели BERT<br>
Текст с лемматизацией для построения векторов TF-IDF в sklearn<br>
Обработанный текс сохранили в  файл `processed_data.csv`<br>


По качеству модели `DistilBertForSequenceClassification` можно сделать вывод, что до обученная модель<br>
На небольшом количестве текстов классифицировала с невысоким качеством f1 `0.731988`<br>
Количество текстов для обучения модели в этом примере было 4000 <br>
Модель обученная на 80% от корпуса текстов показывает лучший результат `0.901617`<br>


Модель `LogisticRegression` показала качество на тестовой выборке удовлетворяющее заданным параметрам `0.774213`<br>
Также стоит отметить, что скорость обучения модели на 80% от всего корпуса с TF-IDF текста во много раз превосходят <br>
Скорость обучения `DistilBertForSequenceClassification`<br>


## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны